In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME =  "Quadcode/src/ai"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Quadcode/src/ai
condacolab_install.log	env.py	__pycache__  Training.ipynb


In [ ]:
!pip install stable-baselines3

In [ ]:
# # install finrl library
# !pip install wrds
# !pip install swig
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [ ]:
# from env import *
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym
    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date
    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step

    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                user_df,
                # hmax,
                initial_amount,
                # transaction_cost_pct,
                state_space,
                action_space,
                tech_indicator_list,
                #turbulence_threshold,
                lookback=252,
                stock_dim=49,
                reward_scaling=0.1,

                profit_weight=0.5,
                sdg_weight=0.5,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.user_df = user_df
        # self.esg_df = esg_df
        self.stock_dim = stock_dim
        # self.hmax = hmax
        self.initial_amount = initial_amount
        # self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, #0
                                            high=10000,
                                            shape = (len(self.tech_indicator_list) + self.state_space , #+6
                                                     self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs),
                      [self.data[tech].values.tolist()for tech in self.tech_indicator_list ], axis=0) #self.data[tech].values.tolist()
        self.terminal = False
        #self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.Date]  #self.data.Date.unique()[0]
        # self.user_sdg_preferences = self.user_df.sample(n=1, random_state=42)[['SDG_' +str(i) for i in range(1,18)]].values
        self.user_sdg_preferences = self.user_df.sample(n=1, random_state=42)[['ESG_' +str(i) for i in range(1,4)]].values

        self.profit_weight= profit_weight
        self.sdg_weight = sdg_weight


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)
        self.user_sdg_preferences = self.user_df.sample(n=1, random_state=42)[['ESG_' +str(i) for i in range(1,4)]].values


        if self.terminal:
            rwd_dict = {"daily_return": self.portfolio_return_memory, "daily_social_impact": self.social_return_memory}
            # df = pd.DataFrame(self.portfolio_return_memory)
            # df.columns = ['daily_return']

            df = pd.DataFrame(rwd_dict)
            print("Culm. Return:", df.daily_return.cumsum())
            # plt.plot(df.daily_return.cumsum(),'r')
            # plt.savefig('results/cumulative_reward.png')
            # plt.close()

            # plt.plot(self.portfolio_return_memory,'r')
            # plt.savefig('results/rewards.png')
            # plt.close()


            # plt.plot(df.daily_social_impact.cumsum(),'s')
            # plt.savefig('results/cumulative_social_impact.png')
            # plt.close()

            # plt.plot(self.portfolio_return_memory,'s')
            # plt.savefig('results/social_impact.png')
            # plt.close()
            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']


            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")



            return self.state, self.reward, self.terminal,{}

        else:
            # actions are the portfolio weight
            # normalize to sum of 1
            norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            weights = norm_actions
            # print(weights)
            # print(np.sum(weights))
            #print(weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data.copy()

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]


            # print(last_day_memory.shape)
            # print(self.data.shape)

            self.covs = self.data['cov_list'].values[0]
            print(self.covs)
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.Close.values / last_day_memory.Close.values)-1)*weights)            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.Date)
            self.asset_memory.append(new_portfolio_value)

            # # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            print(self.reward)

            social_impact = self.calculate_social_alignment(weights)
            print(social_impact)

            #self.reward = self.calculate_reward(new_portfolio_value, weights)
            self.social_return_memory.append(social_impact)

            ### User Preference Reward ####


            #self.reward = self.reward*self.reward_scaling


        return self.state, self.reward, self.terminal, {}



    def calculate_reward(self, portfolio_value,social_impact, weights):
        sdg_scores = self.data['sdg_scores'].values
        sdg_preferences = np.array([self.user_sdg_preferences[sdg] for sdg in self.user_sdg_preferences.keys()])
        sdg_alignment = np.dot(sdg_scores, sdg_preferences)
        reward = self.profit_weight * portfolio_value + self.sdg_weight * sdg_alignment
        return reward

    def calculate_social_alignment(self,weights ):
        sdg_scores = self.data[['ESG_' +str(i) for i in range(1,4)]].values
        print(sdg_scores.shape)
        weighted_sdgs = np.matmul( weights, sdg_scores)
        print(weighted_sdgs)
        print(weighted_sdgs.shape)
        print(self.user_sdg_preferences)
        # sdg_preferences = np.array([self.user_sdg_preferences[sdg] for sdg in self.user_sdg_preferences.keys()])
        sdg_alignment = np.dot(weighted_sdgs, np.array(self.user_sdg_preferences).reshape(3, -1))
        return sdg_alignment

    def get_state(self):
         obs =np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
         obs = np.append(obs, [self.data[tech].values.tolist() for tech in ['ESG_' +str(i) for i in range(1,4)] ],  axis=0)
         duplicated_array = np.tile(self.user_sdg_preferences, (48,1))
         print(obs.shape)
         print(duplicated_array.shape)

         obs = np.append(obs, duplicated_array.T,  axis=0)
        #  obs = np.append(obs.flatten(), self.user_sdg_preferences,  axis=0)
         return obs

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        print(self.state.shape)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.social_return_memory =[0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.Date]
        return self.state

    def render(self, mode='human'):
        return self.state

    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions



    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [ ]:
# Load data
financial_data = pd.read_csv('../real_data/stock_data_ti.csv')
esg_data = pd.read_csv('../real_data/companies_sdgs_vector.csv')
esg_vector = pd.read_csv('../real_data/companies_esg_vector.csv')
info_data = pd.read_csv('../real_data/companies_info.csv')

user_data = pd.read_csv('../real_data/users.csv')
user_esg_data = pd.read_csv('../real_data/users_esg.csv')

financial_data = financial_data.bfill()
financial_data.head(5)

,Unnamed: 0,Date,stock,Adj Close,Close,High,Low,Open,Volume,MACD,MACD_Signal,MACD_Hist,RSI,CCI,ADX
0,0,2018-01-02,AAPL,40.670979,43.064999,43.075001,42.314999,42.540001,102223600.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172
1,47,2018-01-03,AAPL,40.663902,43.057499,43.637501,42.990002,43.132500,118071600.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172
2,94,2018-01-04,AAPL,40.852783,43.257500,43.367500,43.020000,43.134998,89738400.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172
3,141,2018-01-05,AAPL,41.317902,43.750000,43.842499,43.262501,43.360001,94640000.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172
4,188,2018-01-08,AAPL,41.164444,43.587502,43.902500,43.482498,43.587502,82271200.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172


In [ ]:
esg_vector.head()

,Unnamed: 0,stock,longBusinessSummary,overallRisk,environmental,social,governance
0,0,TSLA,"Tesla, Inc. designs, develops, manufactures, l...",10,0,0,1
1,1,NVDA,NVIDIA Corporation provides graphics and compu...,7,1,0,0
2,2,AAPL,"Apple Inc. designs, manufactures, and markets ...",1,0,0,0
3,3,MSFT,Microsoft Corporation develops and supports so...,1,0,0,1
4,4,AMZN,"Amazon.com, Inc. engages in the retail sale of...",9,1,0,0


In [ ]:
user_data.head()

,Unnamed: 0,Username,Desired_Risk,Age,Investment_Amount,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,...,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,ESG_1,ESG_2,ESG_3
0,0,User_1,Low,52,16262.172067,0.0,0.0,0.0,0.2,0.0,...,0.2,0.0,0.2,0.0,0.0,0.2,0.0,0,0,1
1,1,User_2,High,54,84829.915374,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0,1,0
2,2,User_3,Medium,25,32823.605854,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0,1,0
3,3,User_4,Low,64,56007.128253,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,1,0,0
4,4,User_5,Medium,52,54512.068564,0.0,0.0,0.0,0.2,0.0,...,0.2,0.2,0.0,0.0,0.2,0.2,0.0,1,0,0


In [ ]:
user_esg_data.head()

,Unnamed: 0,Username,ESG_1,ESG_2,ESG_3
0,0,User_1,1,0,0
1,1,User_2,0,1,0
2,2,User_3,0,1,0
3,3,User_4,0,1,0
4,4,User_5,1,0,0


In [ ]:
len(info_data['stock'].unique())

49

In [ ]:
info_data['overallRisk'].max()

10

In [ ]:
len(esg_vector['stock'].unique())

49

In [ ]:
# user_data = user_data.merge(user_esg_data, on='Username')

In [ ]:
user_data.head()

,Unnamed: 0,Username,Desired_Risk,Age,Investment_Amount,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,...,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,ESG_1,ESG_2,ESG_3
0,0,User_1,Low,52,16262.172067,0.0,0.0,0.0,0.2,0.0,...,0.2,0.0,0.2,0.0,0.0,0.2,0.0,0,0,1
1,1,User_2,High,54,84829.915374,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0,1,0
2,2,User_3,Medium,25,32823.605854,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0,1,0
3,3,User_4,Low,64,56007.128253,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,1,0,0
4,4,User_5,Medium,52,54512.068564,0.0,0.0,0.0,0.2,0.0,...,0.2,0.2,0.0,0.0,0.2,0.2,0.0,1,0,0


In [ ]:
esg_data = esg_data.rename(columns={"Ticker Symbol": "stock"})

In [ ]:
esg_vector.head()

,Unnamed: 0,stock,longBusinessSummary,overallRisk,environmental,social,governance
0,0,TSLA,"Tesla, Inc. designs, develops, manufactures, l...",10,0,0,1
1,1,NVDA,NVIDIA Corporation provides graphics and compu...,7,1,0,0
2,2,AAPL,"Apple Inc. designs, manufactures, and markets ...",1,0,0,0
3,3,MSFT,Microsoft Corporation develops and supports so...,1,0,0,1
4,4,AMZN,"Amazon.com, Inc. engages in the retail sale of...",9,1,0,0


In [ ]:
financial_data.columns

Index(['Unnamed: 0', 'Date', 'stock', 'Adj Close', 'Close', 'High', 'Low',
       'Open', 'Volume', 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'CCI',
       'ADX'],
      dtype='object')

In [ ]:
esg_vector = esg_vector.rename(columns={"environmental":"ESG_1",	"social":"ESG_2", 	"governance": "ESG_3"})

df = financial_data.merge(esg_vector[['stock', 'ESG_1', 'ESG_2', 'ESG_3','overallRisk']], on='stock')

In [ ]:
df.head()

,Unnamed: 0,Date,stock,Adj Close,Close,High,Low,Open,Volume,MACD,MACD_Signal,MACD_Hist,RSI,CCI,ADX,ESG_1,ESG_2,ESG_3,overallRisk
0,0,2018-01-02,AAPL,40.670979,43.064999,43.075001,42.314999,42.540001,102223600.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172,0,0,0,1
1,47,2018-01-03,AAPL,40.663902,43.057499,43.637501,42.990002,43.132500,118071600.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172,0,0,0,1
2,94,2018-01-04,AAPL,40.852783,43.257500,43.367500,43.020000,43.134998,89738400.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172,0,0,0,1
3,141,2018-01-05,AAPL,41.317902,43.750000,43.842499,43.262501,43.360001,94640000.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172,0,0,0,1
4,188,2018-01-08,AAPL,41.164444,43.587502,43.902500,43.482498,43.587502,82271200.0,-0.442614,-1.021489,0.578875,68.998426,56.696799,33.743172,0,0,0,1


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# # Initialize the StandardScaler
# scaler = MinMaxScaler()



# # Standard scale the DataFrame (excluding the target column if it's not to be scaled)
# features_to_scale = df.drop(columns=[ 	'stock', 	'Unnamed: 0', 	'Date',  'ESG_1', 'ESG_2', 'ESG_3'])
# scaled_features = scaler.fit_transform(features_to_scale)

# # Create a new DataFrame with the scaled features
# scaled_df = pd.DataFrame(scaled_features, columns=features_to_scale.columns)

# # # Optionally, add the target column back to the scaled DataFrame
# # scaled_df['target'] = df['target'].values

# # Display the scaled DataFrame
# print("\nScaled DataFrame:")
# scaled_df.head()

# df[features_to_scale.columns] = scaled_df
# df.head()

In [ ]:
df = df[df['stock'] != 'RIVN']

In [ ]:
# add covariance matrix as states
df=df.sort_values(['Date','stock'],ignore_index=True)
df.index = df.Date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'Date',columns = 'stock', values = 'Close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values
  cov_list.append(covs)

df_cov = pd.DataFrame({'Date':df.Date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='Date')
df = df.sort_values(['Date'])#.reset_index(drop=True)


# Get unique dates and generate a mapping to day_index
unique_dates = pd.Series(df['Date'].unique())
date_to_index = {date: idx for idx, date in unique_dates.items()}

# Map the 'date' column to the new 'day_index'
df['day_index'] = df['Date'].map(date_to_index)

# # Convert 'date' and 'tic' columns to a MultiIndex
# df.set_index(['Date', 'stock'], inplace=True)


# Set the new index with 'day_index' and 'tic'
df.set_index(['day_index', 'stock'], inplace=True)
# Reset the index to make 'day_index' the main index
df.reset_index(inplace=True)

df.head()

,day_index,stock,Unnamed: 0,Date,Adj Close,Close,High,Low,Open,Volume,...,MACD_Signal,MACD_Hist,RSI,CCI,ADX,ESG_1,ESG_2,ESG_3,overallRisk,cov_list
0,0,AAPL,11861,2019-01-03,34.075386,35.547501,36.430000,35.500000,35.994999,365248800.0,...,-2.286904,0.020891,27.883308,-169.031010,46.841128,0,0,0,1,"[[0.0011750413912520832, 0.0009736530936573836..."
1,0,MRNA,11887,2019-01-03,15.500000,15.500000,16.080000,15.200000,15.490000,1588900.0,...,-0.309910,-0.177262,35.334143,-6.369853,14.368347,0,0,1,9,"[[0.0011750413912520832, 0.0009736530936573836..."
2,0,MSFT,11888,2019-01-03,92.153824,97.400002,100.190002,97.199997,100.099998,42579100.0,...,-1.839320,-0.359113,38.895112,-81.784470,21.102374,0,0,1,1,"[[0.0011750413912520832, 0.0009736530936573836..."
3,0,NEE,11889,2019-01-03,37.252155,42.352501,42.790001,42.167500,42.477501,9260800.0,...,-0.278216,-0.216114,37.006889,-88.237358,23.861969,0,0,1,4,"[[0.0011750413912520832, 0.0009736530936573836..."
4,0,NFLX,11890,2019-01-03,271.200012,271.200012,275.790009,264.429993,270.200012,14969600.0,...,-8.916319,2.615053,50.759383,68.542007,23.736368,0,0,1,8,"[[0.0011750413912520832, 0.0009736530936573836..."


In [ ]:
df['stock']

0        AAPL
1        MRNA
2        MSFT
3         NEE
4        NFLX
         ... 
24187     IBM
24188    INTC
24189     JNJ
24190      MA
24191     XOM
Name: stock, Length: 24192, dtype: object

In [ ]:
df.set_index(['day_index'], inplace=True)

In [ ]:
df.head()

,stock,Unnamed: 0,Date,Adj Close,Close,High,Low,Open,Volume,MACD,MACD_Signal,MACD_Hist,RSI,CCI,ADX,ESG_1,ESG_2,ESG_3,overallRisk,cov_list
day_index,,,,,,,,,,,,,,,,,,,,
0,AAPL,11861,2019-01-03,34.075386,35.547501,36.430000,35.500000,35.994999,365248800.0,-2.266013,-2.286904,0.020891,27.883308,-169.031010,46.841128,0,0,0,1,"[[0.0011750413912520832, 0.0009736530936573836..."
0,MRNA,11887,2019-01-03,15.500000,15.500000,16.080000,15.200000,15.490000,1588900.0,-0.487172,-0.309910,-0.177262,35.334143,-6.369853,14.368347,0,0,1,9,"[[0.0011750413912520832, 0.0009736530936573836..."
0,MSFT,11888,2019-01-03,92.153824,97.400002,100.190002,97.199997,100.099998,42579100.0,-2.198433,-1.839320,-0.359113,38.895112,-81.784470,21.102374,0,0,1,1,"[[0.0011750413912520832, 0.0009736530936573836..."
0,NEE,11889,2019-01-03,37.252155,42.352501,42.790001,42.167500,42.477501,9260800.0,-0.494331,-0.278216,-0.216114,37.006889,-88.237358,23.861969,0,0,1,4,"[[0.0011750413912520832, 0.0009736530936573836..."
0,NFLX,11890,2019-01-03,271.200012,271.200012,275.790009,264.429993,270.200012,14969600.0,-6.301266,-8.916319,2.615053,50.759383,68.542007,23.736368,0,0,1,8,"[[0.0011750413912520832, 0.0009736530936573836..."


In [ ]:
df.loc[0,:]['Close'].shape

(48,)

In [ ]:
stocks = df.loc[0,:]['stock'].unique()

In [ ]:
df.loc[0,:]['cov_list'].values[0].shape

(48, 48)

In [ ]:
df['cov_list'].values[0].shape

(48, 48)

In [ ]:
df.shape

(24192, 20)

In [ ]:
user_data.isna().sum()

Unnamed: 0           0
Username             0
Desired_Risk         0
Age                  0
Investment_Amount    0
SDG_1                0
SDG_2                0
SDG_3                0
SDG_4                0
SDG_5                0
SDG_6                0
SDG_7                0
SDG_8                0
SDG_9                0
SDG_10               0
SDG_11               0
SDG_12               0
SDG_13               0
SDG_14               0
SDG_15               0
SDG_16               0
SDG_17               0
ESG_1                0
ESG_2                0
ESG_3                0
dtype: int64

In [ ]:
# # Ensure financial data is sorted by date and company
# financial_data = financial_data.sort_values(by=['Date', 'stock']).reset_index(drop=True)

# Example user preferences for SDGs (these should sum up to 1 or less)
user_preferences = {
    'SDG_1': 0.02, 'SDG_2': 0.88, 'SDG_3': 0.02, 'SDG_4': 0.02,
    'SDG_5': 0.00, 'SDG_6': 0.02, 'SDG_7': 0.02, 'SDG_8': 0.02,
    'SDG_9': 0.02, 'SDG_10': 0.02, 'SDG_11': 0.02, 'SDG_12': 0.02,
    'SDG_13': 0.02, 'SDG_14': 0.02, 'SDG_15': 0.02, 'SDG_16': 0.02,
   'SDG_17': 0.02
}

initial_amount = 100000

tech_indicator_list = ['MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'CCI','ADX']


# Initialize environment
env = StockPortfolioEnv(df, user_data, initial_amount, 48, 48, tech_indicator_list)



# Test the environment
obs = env.reset()
for _ in range(10):
    action = env.action_space.sample()
    # print(action.shape)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        break

(48, 48)
225650.98311115993
(48, 3)
[0.23857252 0.2758436  0.34564509]
(3,)
[[0 1 0]]
[0.2758436]
1378848.4367889473
(48, 3)
[0.24785265 0.24894992 0.43231921]
(3,)
[[0 1 0]]
[0.24894992]
7050806.208199137
(48, 3)
[0.31748883 0.21947125 0.32972362]
(3,)
[[0 1 0]]
[0.21947125]
29034965.166587736
(48, 3)
[0.27721685 0.2765679  0.33264044]
(3,)
[[0 1 0]]
[0.2765679]
121242225.01169892
(48, 3)
[0.25793157 0.29131593 0.2778041 ]
(3,)
[[0 1 0]]
[0.29131593]
211243685.16291833
(48, 3)
[0.32592411 0.1855179  0.35152008]
(3,)
[[0 1 0]]
[0.1855179]
830622571.2879567
(48, 3)
[0.33364208 0.27477084 0.28731704]
(3,)
[[0 1 0]]
[0.27477084]
4552554395.574018
(48, 3)
[0.24020923 0.28819911 0.33773942]
(3,)
[[0 1 0]]
[0.28819911]
13834078911.476635
(48, 3)
[0.2528864  0.21550994 0.43037657]
(3,)
[[0 1 0]]
[0.21550994]
34982004495.72053
(48, 3)
[0.29328069 0.23063718 0.3367578 ]
(3,)
[[0 1 0]]
[0.23063718]


In [ ]:
env.get_state().shape

(57, 48)
(48, 3)


(60, 48)

<h2> AI Modelling </h2>


In [ ]:
# Split the DataFrame
train_df = df.loc[3000:]
test_df = df.loc[:2999]


In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv

# Initialize environment
train_env = StockPortfolioEnv(df, user_data, initial_amount, 48, 48, tech_indicator_list)
train_env = DummyVecEnv([lambda:  StockPortfolioEnv(df, user_data, initial_amount, 48, 48, tech_indicator_list)])


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [ ]:
from stable_baselines3 import PPO

# Initialize the model
model = PPO('MlpPolicy', train_env, verbose=1)

# Train the model
model.learn(total_timesteps=1)

Using cpu device
(54, 48)
[[1.25647897e-03 1.07201687e-03 1.05596011e-03 ... 2.36940495e-04
  3.94736350e-04 5.25028676e-04]
 [1.07201687e-03 1.34407052e-03 1.20637420e-03 ... 1.90852476e-04
  4.86884921e-04 5.60908101e-04]
 [1.05596011e-03 1.20637420e-03 1.35233017e-03 ... 2.08188878e-04
  4.85510793e-04 6.20318973e-04]
 ...
 [2.36940495e-04 1.90852476e-04 2.08188878e-04 ... 2.56838249e-04
  5.85975414e-05 1.87920798e-04]
 [3.94736350e-04 4.86884921e-04 4.85510793e-04 ... 5.85975414e-05
  3.08602777e-04 2.94527344e-04]
 [5.25028676e-04 5.60908101e-04 6.20318973e-04 ... 1.87920798e-04
  2.94527344e-04 5.17732244e-04]]
196711.51026068427
(48, 3)
[0.23573262 0.36748912 0.30215054]
(3,)
[[0 1 0]]
[0.36748912]
[[1.18762559e-03 1.01598741e-03 1.00525730e-03 ... 2.24910404e-04
  3.75374775e-04 4.98051872e-04]
 [1.01598741e-03 1.28248075e-03 1.16888103e-03 ... 1.84452558e-04
  4.69360615e-04 5.37874071e-04]
 [1.00525730e-03 1.16888103e-03 1.34385406e-03 ... 2.06110348e-04
  4.80038636e-04 6.0

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.1

[0.38357138 0.23364331 0.24080485]
(3,)
[[0 1 0]]
[0.23364331]
[[4.33339884e-04 2.54845247e-04 3.04323808e-04 ... 1.02029112e-04
  6.10503887e-05 1.21272048e-04]
 [2.54845247e-04 3.57377214e-04 2.93212907e-04 ... 8.33212741e-05
  1.05258371e-04 1.22147940e-04]
 [3.04323808e-04 2.93212907e-04 4.40901244e-04 ... 9.80546487e-05
  1.13843542e-04 1.37885770e-04]
 ...
 [1.02029112e-04 8.33212741e-05 9.80546487e-05 ... 2.63908627e-04
  2.87476042e-05 7.19933527e-05]
 [6.10503887e-05 1.05258371e-04 1.13843542e-04 ... 2.87476042e-05
  1.22331909e-04 5.74038114e-05]
 [1.21272048e-04 1.22147940e-04 1.37885770e-04 ... 7.19933527e-05
  5.74038114e-05 1.69220531e-04]]
2.6922926785939747e+45
(48, 3)
[0.32316849 0.2936525  0.27875685]
(3,)
[[0 1 0]]
[0.2936525]
[[4.30351167e-04 2.54131188e-04 3.03016817e-04 ... 9.95103274e-05
  6.00005494e-05 1.20884707e-04]
 [2.54131188e-04 3.56658009e-04 2.92888260e-04 ... 8.00827722e-05
  1.03521461e-04 1.22178786e-04]
 [3.03016817e-04 2.92888260e-04 4.39007277e-04

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.1

[0.19798775 0.30730811 0.26210375]
(3,)
[[0 1 0]]
[0.30730811]
[[4.14488284e-04 2.50282521e-04 2.86857623e-04 ... 1.06716119e-04
  5.41158625e-05 1.22125017e-04]
 [2.50282521e-04 3.64547847e-04 2.98980875e-04 ... 1.00182132e-04
  9.82992972e-05 1.14810490e-04]
 [2.86857623e-04 2.98980875e-04 4.25687309e-04 ... 1.00387736e-04
  1.04037564e-04 1.28157003e-04]
 ...
 [1.06716119e-04 1.00182132e-04 1.00387736e-04 ... 2.88495938e-04
  2.69352004e-05 6.80047017e-05]
 [5.41158625e-05 9.82992972e-05 1.04037564e-04 ... 2.69352004e-05
  1.13884693e-04 5.31188383e-05]
 [1.22125017e-04 1.14810490e-04 1.28157003e-04 ... 6.80047017e-05
  5.31188383e-05 1.69095365e-04]]
3.7286602141344723e+50
(48, 3)
[0.36616948 0.1654007  0.37654037]
(3,)
[[0 1 0]]
[0.1654007]
[[4.16420154e-04 2.50496722e-04 2.90211702e-04 ... 1.10917939e-04
  5.53306643e-05 1.21939823e-04]
 [2.50496722e-04 3.62538094e-04 2.98962289e-04 ... 1.01518134e-04
  9.82020812e-05 1.14219701e-04]
 [2.90211702e-04 2.98962289e-04 4.28325148e-04

...

[[0.00085023 0.00068518 0.00049733 ... 0.00040874 0.00031166 0.00044394]
 [0.00068518 0.00091172 0.00055386 ... 0.00038326 0.00031079 0.00034842]
 [0.00049733 0.00055386 0.00059075 ... 0.00021852 0.00021404 0.0001839 ]
 ...
 [0.00040874 0.00038326 0.00021852 ... 0.00085132 0.00014964 0.00044543]
 [0.00031166 0.00031079 0.00021404 ... 0.00014964 0.00039636 0.00015586]
 [0.00044394 0.00034842 0.0001839  ... 0.00044543 0.00015586 0.00111059]]
1.4336801626480743e+179
(48, 3)
[0.2361855  0.30261025 0.28013181]
(3,)
[[0 1 0]]
[0.30261025]
[[0.00085833 0.00068239 0.00049721 ... 0.00041098 0.00031132 0.00044715]
 [0.00068239 0.00091104 0.00055373 ... 0.00038227 0.00031077 0.00034679]
 [0.00049721 0.00055373 0.00059073 ... 0.00021846 0.00021403 0.00018379]
 ...
 [0.00041098 0.00038227 0.00021846 ... 0.00085191 0.00014953 0.00044625]
 [0.00031132 0.00031077 0.00021403 ... 0.00014953 0.00039636 0.00015568]
 [0.00044715 0.00034679 0.00018379 ... 0.00044625 0.00015568 0.00111169]]
3.116075023266541

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(
/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:58: RuntimeWarning: overflow encountered in cast
  obs, self.buf_rews[env_idx], terminated, truncated, self.buf_infos[env_idx] = self.envs[env_idx].step(


[[0.00086573 0.00068108 0.00049764 ... 0.00042361 0.00031317 0.00044453]
 [0.00068108 0.00091399 0.00055251 ... 0.00038533 0.00031081 0.00034408]
 [0.00049764 0.00055251 0.00058853 ... 0.00022692 0.00021517 0.00018394]
 ...
 [0.00042361 0.00038533 0.00022692 ... 0.00086236 0.00015075 0.00044653]
 [0.00031317 0.00031081 0.00021517 ... 0.00015075 0.00039663 0.00015719]
 [0.00044453 0.00034408 0.00018394 ... 0.00044653 0.00015719 0.00111558]]
9.222901547480651e+181
(48, 3)
[0.19544581 0.35896982 0.37066579]
(3,)
[[0 1 0]]
[0.35896982]
Culm. Return: 0        0.000000
1        1.837785
2        5.546968
3        6.251581
4       10.179441
          ...    
499    812.901938
500    813.036758
501    813.407770
502    814.958131
503    816.194909
Name: daily_return, Length: 504, dtype: float64
begin_total_asset:100000
end_total_asset:9.222901547480651e+181
Sharpe:  14.519202069816087
(54, 48)
[[1.25647897e-03 1.07201687e-03 1.05596011e-03 ... 2.36940495e-04
  3.94736350e-04 5.25028676e-04]
 [

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:433: RuntimeWarning: invalid value encountered in multiply
  last_gae_lam = delta + self.gamma * self.gae_lambda * next_non_terminal * last_gae_lam


ValueError: Expected parameter loc (Tensor of shape (64, 48)) of distribution Normal(loc: torch.Size([64, 48]), scale: torch.Size([64, 48])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<AddmmBackward0>)